- Reading multiple json files from external data
- Creating hierarchy folder structure according to country, city, term.
- Distributing all json files under the right subfloder.

In [29]:
from glob import glob
import json
import os

In [30]:
def read_files(file_name):      # Creating a function to read files. 
    with open (file_name, encoding='utf-8', errors="ignore") as file:   # Need to deal with unicode problem.
        content = json.load(file)
        hierarchy_list = {'country': content['location']['country'],'city': content['location']['city'],\
                          'term': content['term'],'name': content['name'].replace(',',' ')}  # Removing the ',' in some names for later output csv file.
        return hierarchy_list

In [90]:
def file_distribution(dict, file):   # Generating hierarchy folder structure and dump json file
    file_name = 'Data Process/' + dict['term'] + '/' + dict['country'] + '/' + dict['city'] + '/' + dict['name'] \
                + '.json'
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    fp = open (file_name, 'w')
    with open (file, 'r', encoding='utf-8', errors="ignore") as jfile:
        fp.write(json.dumps(json.load(jfile)))     
    fp.close()

In [91]:
json_files_path = glob('Data/*.json')     # Calling functions and run get the folder structure.
for json_file in json_files_path:
    file_distribution(read_files(json_file),json_file)

- Reading all restuarants json files.
- Getting all required keys and values.
- Generating a CSV by reading json files with all the restaurant details (name, location, time of operation, etc)

In [81]:
import csv

In [100]:
# Reading file from hierarchy floder 'restaurants' 
def find_files_path(file_root):
    path_list = []
    for root,dirs,files in os.walk(file_root):
        for filepath in files:
            if filepath.endswith('.json'):   
                path_list.append(os.path.join(root,filepath)) 
    return path_list

In [123]:
# Reading each restaurant_file and get useful information
def read_restaurant_files(file_name):
    open_hour_list = []
    with open (file_name, encoding='utf-8', errors="ignore") as file:  
        content = json.load(file)
        if 'hours' in content.keys():
            for i in range(len(content['hours'][0]['open'])):
                open_hour_list.append([content['name'].replace(',',' '), content['location']['country'], \
                                      content['location']['city'], content['hours'][0]['open'][i]['day'], \
                                      content['hours'][0]['open'][i]['start'][:2], content['hours'][0]['open'][i]['start'][-2:], \
                                      content['hours'][0]['open'][i]['end'][:2], content['hours'][0]['open'][i]['end'][-2:]])
    return open_hour_list

In [124]:
# Generating the csv file.
def create_csv_file(list,file):
    with open(file,'w') as f:
        w = csv.writer(f)
        w.writerow(['RESTAURANT NAME','COUNTRY','CITY','DAYS OF WEEK','START TIME(HOUR)','START TIME(MINUTE)',\
                    'END TIME(HOUR)','END TIME(MINUTE)'])
        for cell in list:
            w.writerows(cell)

In [125]:
# Parsing the arguments get final list of restaurants open hours.
whole_list = []
restaurants_path = glob('Data Process/restaurants')
for jfile in find_files_path(restaurants_path[0]):
    whole_list.append(read_restaurant_files(jfile))

In [126]:
# Creating csv file.
create_csv_file(whole_list, 'Restaurants_Open_Hour.csv')